In [1]:
"""

The data hub registration and usage: https://scihub.copernicus.eu/userguide/WebHome

Main API currently only for downloading Sentinel 1/2/3 data
https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/
https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi
https://sentinel.esa.int/documents/247904/685211/Sentinel-2-Products-Specification-Document
"""

import os
import shutil
import time
import zipfile
from pathlib import Path
from datetime import date

import pandas as pd
import geopandas as gpd
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from shapely.geometry import box, Polygon
import numpy as np
path_cur = os.path.abspath('.')
from os.path import dirname as up

In [7]:
path_cur

'/rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/scripts'

In [2]:
base_path = Path(os.path.join(up(path_cur), 'raw_data', 'sentinel_shoreline'))

(base_path / 'sentinel' / 'tmp').mkdir(exist_ok=True, parents=True)
(base_path / 'sentinel' / 'compressed').mkdir(exist_ok=True, parents=True)
(base_path / 'sentinel' / 'uncompressed').mkdir(exist_ok=True, parents=True)

In [6]:
### No
# S2A_MSIL1C_20170519T161221_N0205_R097_T18SUH_20170519T161218.SAFE
# S2A_MSIL1C_20170529T155911_N0205_R097_T18SUG_20170529T160140.SAFE
### 

### Yes
# S2B_MSIL1C_20170730T154909_N0205_R054_T18SUJ_20170730T160022.SAFE
# S2B_MSIL1C_20170928T155019_N0205_R054_T18SUF_20170928T155933.SAFE
###


# 2016, 2017
# 2018, 2019
year = 2017
temporal_start = '{}0501'.format(year)
temporal_end = '{}1230'.format(year)

platformname = 'Sentinel-2'
processinglevel = 'Level-2A' #1C

max_cloud_percent = 25

user = 'mirandalv12'
password = '!u@EfpYBYmWL7W5'


# api_url = 'https://scihub.copernicus.eu/dhus'
api_url = 'https://apihub.copernicus.eu/apihub/'

def get_poly_bound(gdf):
    
    bounds = gdf['geometry'].bounds
    minx = np.min(bounds.minx)
    miny = np.min(bounds.miny)
    maxx = np.max(bounds.maxx)
    maxy = np.max(bounds.maxy)
    
    return minx, miny, maxx, maxy


# boundary_file = os.path.join(up(path_cur), 'data', 'sir2005-5073_shed_shape', 'cbshed_boundary.geojson')
boundary_file = os.path.join(up(path_cur), 'data', 'TMI_2011_2019', 'VA_TMI_2011_2019.shp')

boundary_df = gpd.read_file(boundary_file)
boundary_df = boundary_df.to_crs(4326)
bounds = boundary_df['geometry'].bounds

minx, miny, maxx, maxy = get_poly_bound(boundary_df)
new_poly = box(minx, miny, maxx, maxy)

new_poly_geojson = gpd.GeoSeries([new_poly]).__geo_interface__
footprint = geojson_to_wkt(new_poly_geojson)

# # # search by polygon, time, and SciHub query keywords
# footprint = geojson_to_wkt(read_geojson(boundary_file))


api = SentinelAPI(user, password, api_url)
products = api.query(footprint,
                     date=(temporal_start, temporal_end),
                     platformname=platformname,
                     processinglevel=processinglevel,
                     cloudcoverpercentage=(0, max_cloud_percent))




In [237]:
# convert to Pandas DataFrame
df = api.to_dataframe(products)
df = df[df['tileid'].notnull()]

df_path = base_path / 'sentinel_query_{}_{}.csv'.format(year, processinglevel)

# save if running new query
df.to_csv(df_path, index=False, encoding="utf-8")

# load if query was already run
df = pd.read_csv(df_path)

# sort and limit to first 5 sorted products
# df_sorted = df.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])
# df_sorted = df_sorted.head(5)

# api.download_all(df_sorted.uuid)

tile_list = ['18STH', '18STG', '18SUJ', '18SUH', '18SUG', '18SUF', '18SVH', '18SVG', '18SVF', '18STJ', '18SVJ']
df = df[df['tileid'].isin(tile_list)]

In [235]:
# subtile_list = ['18STJ', '18SVJ'] #'18SUJ', '18SUH', '18SUG', '18SUF']
# df = df[df['tileid'].isin(subtile_list)]

In [238]:
# df_sorted = df.sort_values('cloudcoverpercentage')
# # api.download_all(df_sorted.uuid)
# # df_sorted[df_sorted['cloudcoverpercentage']==0.0117].uuid # [3bd2224a-ed87-40a9-89e6-ad86675b18ba, 65299e00-86a3-4316-822e-c9a0c1460c7d]
# df_sorted[df_sorted['cloudcoverpercentage']==0.2274].uuid
needlist = ['3bd2224a-ed87-40a9-89e6-ad86675b18ba', '65299e00-86a3-4316-822e-c9a0c1460c7d']

In [239]:
df_sorted = df[df['uuid'].isin(needlist)]


,title,link,link_alternative,link_icon,summary,ondemand,datatakesensingstart,beginposition,endposition,ingestiondate,...,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier
89,S2A_MSIL1C_20170804T154911_N0205_R054_T18STJ_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2017-08-04T15:49:11.026Z, Instrument: MS...",False,2017-08-04 15:49:11.026,2017-08-04 15:49:11.026,2017-08-04 15:49:11.026,2017-09-02 08:01:46.785,...,S2MSI1C,2015-000A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20170804T154911_N0205_R054_T18STJ_2...,65299e00-86a3-4316-822e-c9a0c1460c7d,S2A_OPER_MSI_L1C_TL_EPA__20170901T014051_A0110...,S2A_OPER_MSI_L1C_TL_EPA__20170901T014051_A0110...,S2A_OPER_MSI_L1C_DS_EPA__20170901T014051_S2017...
162,S2B_MSIL1C_20170730T154909_N0205_R054_T18SVJ_2...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,https://apihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2017-07-30T15:49:09.027Z, Instrument: MS...",False,2017-07-30 15:49:09.027,2017-07-30 15:49:09.027,2017-07-30 15:49:09.027,2017-07-31 02:32:09.899,...,S2MSI1C,2015-000A,DESCENDING,Sentinel-2B,Level-1C,S2B_MSIL1C_20170730T154909_N0205_R054_T18SVJ_2...,3bd2224a-ed87-40a9-89e6-ad86675b18ba,S2B_OPER_MSI_L1C_TL_MTI__20170730T210039_A0020...,S2B_OPER_MSI_L1C_TL_MTI__20170730T210039_A0020...,S2B_OPER_MSI_L1C_DS_MTI__20170730T210039_S2017...


In [240]:
api.download_all(df_sorted.uuid)

LTA retrieval:   0%|          | 0/1 [00:00<?, ?product/s]

MD5 checksumming:   0%|          | 0.00/799M [00:00<?, ?B/s]

MD5 checksumming:   0%|          | 0.00/433M [00:00<?, ?B/s]

ResultTuple(downloaded={'65299e00-86a3-4316-822e-c9a0c1460c7d': {'id': '65299e00-86a3-4316-822e-c9a0c1460c7d', 'title': 'S2A_MSIL1C_20170804T154911_N0205_R054_T18STJ_20170804T155116', 'size': 433198259, 'md5': '151344df6cc0b3d4e363748f3dd2da85', 'date': datetime.datetime(2017, 8, 4, 15, 49, 11, 26000), 'footprint': 'POLYGON((-77.95501618949858 38.721873240450215,-77.95066562228976 38.736650112670674,-77.90736697802352 38.8840438015932,-77.86381084178393 39.031435911609634,-77.82003525066794 39.17879391197132,-77.77641804996217 39.32616990953295,-77.73262490020038 39.47349243900279,-77.68872825873491 39.620751061226336,-77.65960703858204 39.71819808181771,-77.21960079191267 39.7290432642928,-77.18865192655385 38.74036995699254,-77.95501618949858 38.721873240450215))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('65299e00-86a3-4316-822e-c9a0c1460c7d')/$value", 'Online': True, 'Creation Date': datetime.datetime(2017, 9, 2, 8, 6, 50, 672000), 'Ingestion Date': datetime.da

In [148]:
df["downloaded"] = 0

def set_download_granules(df, error_title=None):
    if error_title is not None:
        df.loc[df.title == error_title, "downloaded"] = -1
    for i in set(df["tileid"].to_list()):
        best_title = df.loc[(df.tileid == i) & (df.downloaded != -1)].sort_values("cloudcoverpercentage").iloc[4].title
        df.loc[df.title == best_title, "downloaded"] = 1
    return df


def is_bad_zip(x):
    """ Return true if zipfile is bad
    """
    try:
        test = zipfile.ZipFile(x).testzip()
        if test is not None:
            return test
        else:
            return False
    except Exception as e:
        return e

# best_df = join_df.loc[join_df.groupby("Name").cloudcoverpercentage.idxmin()]
# best_df.cloudcoverpercentage.describe()
# best_df.reset_index(drop=True, inplace=True)

df = set_download_granules(df, error_title=None)

dl_base_path = base_path / 'sentinel'

dl_tmp_path = os.path.join(dl_base_path, "tmp")
dl_final_path = os.path.join(dl_base_path, "compressed")



# remove bad downloads from tmp dir if needed
for i in df.loc[df.downloaded == -1].title:
    f = os.path.join(dl_base_path, "tmp", i + ".zip")
    if os.path.isfile(f):
        print(i)
        os.remove(f)


# remove entire tmp dir and recreate
shutil.rmtree(dl_tmp_path)
os.makedirs(dl_tmp_path)


# # remove downloads from compressed dir if no longer in current download list
# current_contents = [os.path.splitext(i)[0] for i in os.listdir(dl_final_path)]
# old_content = [i for i in current_contents if i not in df.loc[df.downloaded == 1].title.to_list()]

# for i in old_content:
#     print("Removing", i)
#     os.remove(os.path.join(dl_final_path, i + ".zip"))


# get rid of any bad zips from previous runs
# (mainly for removing bad zips from before code checked zip immediately after download)
best_df = df.loc[df.downloaded == 1]
for i in range(len(best_df)):
    base_name = best_df.title.iloc[i] + ".zip"
    final_path = os.path.join(dl_final_path, base_name)
    if os.path.isfile(final_path) and is_bad_zip(final_path):
        print("Removing", best_df.title.iloc[i])
        # os.remove(final_path)
        # join_df = set_download_granules(join_df, error_title=best_df.title.iloc[i])


# bad_titles = []

# not_coming_online = []

# for i in bad_titles + not_coming_online:
#     df = set_download_granules(df, i)



# to_download = []

# current_contents = [os.path.splitext(i)[0] for i in os.listdir(dl_final_path)]

# df.loc[df.title.isin(current_contents + to_download), "downloaded"] = 1
# df.loc[df.title.isin(bad_titles + not_coming_online), "downloaded"] = -1


# for i, row in df.loc[(df.downloaded == 1) & df.tileid.isin(["51PWL","51NYH"])].iterrows():
#     print(row.title, row.cloudcoverpercentage)

# set_zero = []

# df.loc[df.title.isin(set_zero), "downloaded"] = 0


# df.loc[df.downloaded == 1]

# df.loc[df.downloaded == 1].tileid.value_counts()

# df.loc[df.downloaded == 1].cloudcoverpercentage.describe()



In [149]:
df.loc[df.downloaded == 1].cloudcoverpercentage.describe()

count    1.0000
mean     0.0959
std         NaN
min      0.0959
25%      0.0959
50%      0.0959
75%      0.0959
max      0.0959
Name: cloudcoverpercentage, dtype: float64

In [150]:
# set to true for initial run where data may need to be retrieved from
# the long term archive (lta). If you know data is available, set to
# false to allow for faster downloads
lta_query = True

any_offline = True

while any_offline:
    print("Restarting download checks...")
    any_offline = False
    best_df = df.loc[df.downloaded == 1]
    existing_files = os.listdir(dl_final_path)
    for i in range(len(best_df)):
        base_name = best_df.title.iloc[i] + ".zip"
        tmp_path = os.path.join(dl_tmp_path, base_name)
        final_path = os.path.join(dl_final_path, base_name)
        # if is_bad_zip(final_path):
        #     os.remove(final_path)
        if base_name not in existing_files:
        # if not os.path.isfile(final_path):
            print("\t{}/{} - {}".format(i+1, len(best_df), best_df.title.iloc[i]))
            try:
                tmp = api.download(best_df.uuid.iloc[i], directory_path=dl_tmp_path)
                if tmp["Online"]:
                    zip_error = is_bad_zip(tmp_path)
                    if zip_error:
                        print("\t\tbad zip:", zip_error)
                        df = set_download_granules(df, error_title=best_df.title.iloc[i])
                        os.remove(tmp_path)
                        any_offline = True
                    else:
                        shutil.move(tmp_path, final_path)
            except Exception as e:
                print("API Error:", best_df.title.iloc[i], e)
                # raise
                print(best_df.title.iloc[i])
                df = set_download_granules(df, error_title=best_df.title.iloc[i])
                any_offline = True
            else:
                any_offline = True
            if lta_query:
                time.sleep(60)




granules_df_path = base_path / 'sentinel/sentinel_granules.csv'

# save if running new query
df.to_csv(granules_df_path, index=False, encoding="utf-8")

df = pd.read_csv(granules_df_path)



Restarting download checks...
	1/1 - S2A_MSIL1C_20170804T154911_N0205_R054_T18SUJ_20170804T155116
API Error: S2A_MSIL1C_20170804T154911_N0205_R054_T18SUJ_20170804T155116 Product 72138029-8d0a-4be0-93d3-aa264a71d001 is not online. Triggered retrieval from the Long Term Archive.
S2A_MSIL1C_20170804T154911_N0205_R054_T18SUJ_20170804T155116


KeyboardInterrupt: 

In [76]:
"""
# other api functions:

# download all results from the search
api.download_all(df.uuid, directory_path=dl_path, n_concurrent_dl=3)

# GeoJSON FeatureCollection containing footprints and metadata of the scenes
api.to_geojson(products)

# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
api.to_geodataframe(products)

# Get basic information about the product: its title, file size, MD5 sum, date, footprint and
# its download url
api.get_product_odata("<product_id>")

# Get the product's full metadata available on the server
api.get_product_odata("<product_id>", full=True)

"""

'\n# other api functions:\n\n# download all results from the search\napi.download_all(df.uuid, directory_path=dl_path, n_concurrent_dl=3)\n\n# GeoJSON FeatureCollection containing footprints and metadata of the scenes\napi.to_geojson(products)\n\n# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries\napi.to_geodataframe(products)\n\n# Get basic information about the product: its title, file size, MD5 sum, date, footprint and\n# its download url\napi.get_product_odata("<product_id>")\n\n# Get the product\'s full metadata available on the server\napi.get_product_odata("<product_id>", full=True)\n\n'

In [241]:
import os
from zipfile import ZipFile
import glob
import multiprocessing

year = '2017'

zip_list = glob.glob('/rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/raw_data/sentinel_shoreline/sentinel/compressed/*_*_{}*'.format(year))

uncompressed_dir = base_path / 'sentinel/uncompressed'


def unzip(zip_path, overwrite=False):
    base_name = os.path.basename(zip_path).split('.')[0]
    print("Unzipping", base_name)
    safe_name = base_name + ".SAFE"
    safe_dir = os.path.join(uncompressed_dir, safe_name)
    exists = os.path.isdir(safe_dir)
    if overwrite and exists:
        print("\tOverwritting")
        zipObj = ZipFile(zip_path, 'r')
        zipObj.extractall(uncompressed_dir)
    elif not exists:
        print("\tWritting")
        zipObj = ZipFile(zip_path, 'r')
        zipObj.extractall(uncompressed_dir)
    else:
        print("\tSkipping existing")

with multiprocessing.Pool(2) as p:
    p.map(unzip, zip_list)

for i in zip_list:
    try:
        x = ZipFile(i)
    except:
        print(i)



Unzipping S2A_MSIL1C_20170509T155911_N0205_R097_T18STH_20170509T160214
Unzipping S2B_MSIL1C_20170928T155019_N0205_R054_T18SVF_20170928T155933
	Skipping existing
	Skipping existing
Unzipping S2A_MSIL1C_20170913T154901_N0205_R054_T18STF_20170913T155123
Unzipping S2A_MSIL1C_20170516T154911_N0205_R054_T18SVH_20170516T160105
	Writting
	Skipping existing
Unzipping S2A_MSIL1C_20170516T154911_N0205_R054_T18SUH_20170516T160105
	Skipping existing
Unzipping S2B_MSIL1C_20170730T154909_N0205_R054_T18SUJ_20170730T160022
	Skipping existing
Unzipping S2B_MSIL1C_20170908T154859_N0205_R054_T18SUF_20170908T155147
	Skipping existing
Unzipping S2B_MSIL1C_20170928T155019_N0205_R054_T18SUG_20170928T155933
	Skipping existing
Unzipping S2B_MSIL1C_20170630T160029_N0205_R054_T18SVG_20170630T160032
	Skipping existing
Unzipping S2B_MSIL1C_20170730T154909_N0205_R054_T18STG_20170730T160022
	Writting
Unzipping S2A_MSIL1C_20170804T154911_N0205_R054_T18STJ_20170804T155116
	Writting
Unzipping S2B_MSIL1C_20170730T154909_

In [242]:

uncompressed_list = [os.path.splitext(os.path.basename(i))[0] for i in glob.glob('/rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/raw_data/sentinel_shoreline/sentinel/uncompressed/*_*_{}*'.format(year))]
compressed_list = [os.path.splitext(os.path.basename(i))[0] for i in glob.glob('/rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/raw_data/sentinel_shoreline/sentinel/compressed/*_*_{}*'.format(year))]

extra_uncompressed_list = [i for i in uncompressed_list if i not in compressed_list]
missing_compressed_list = [i for i in compressed_list if i not in uncompressed_list]

for i in extra_uncompressed_list:
    print(i)
    extra_dir = str(base_path) + '/sentinel/uncompressed/' + i + '.SAFE'
#     shutil.rmtree(extra_dir)


S2A_MSIL1C_20170913T154901_N0205_R054_T18STF_20200802T082733


In [243]:
base_path

PosixPath('/rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/raw_data/sentinel_shoreline')

In [244]:
def re_projection(intif, outtif):
        
    dst_crs = 'EPSG:4326'
    
    src = rasterio.open(intif, driver='JP2OpenJPEG')
    transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    
    with rasterio.open(outtif, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
    


In [245]:
# =============================================================================

import glob
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from osgeo import gdal

b_path = "/rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/raw_data/sentinel_shoreline"

(base_path / 'sentinel' / 'reprojection').mkdir(exist_ok=True, parents=True)
reproject_path = os.path.join(b_path, 'sentinel','reprojection')

band_list = ["01", "02", "03", "04", "05", "06", "07", "08", "8A", "09", "10", "11", "12"]

# for b in band_list:
# b = "03"

for b in band_list:
    
    print("Working on band {}".format(b))
    
    band_granules_path = glob.glob(os.path.join(b_path, 'sentinel/uncompressed/*_*_{}*/GRANULE/*/IMG_DATA/*_B{}.jp2'.format(year, b)))

    band_granules_src = [rasterio.open(i, driver='JP2OpenJPEG') for i in band_granules_path]
    proj_band_path = [os.path.join(reproject_path, Path(i).stem + '_wgs84.tif') for i in band_granules_path]

    # Reproject all bands to wgs-84 before mosaic
    for file,outfile in zip(band_granules_path, proj_band_path):

        infilename = os.path.basename(file).split('.')[0]
        outfilename_pre = os.path.basename(outfile).split('.')[0][0:-6]

        if infilename == outfilename_pre and not os.path.isfile(outfile):
             
             print("Reproject file {}".format(file))
             
             re_projection(file, outfile)

    print("proj_band_path is {}".format(proj_band_path))
    # Create mosaic band
    
    print("Start mosaic band {} in year {}".format(b, year))
    mosaic_path = base_path / 'sentinel/merge_B{}_{}.tif'.format(b, year)
    
    if not os.path.isfile(mosaic_path):

        new_band_granules_src = [rasterio.open(i) for i in proj_band_path]

        mosaic, output = merge(new_band_granules_src)

        output_meta = new_band_granules_src[0].meta.copy()
        output_meta.update(
                {"driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": output,
                }
        )

        with rasterio.open(mosaic_path, "w", **output_meta) as m:
            m.write(mosaic)


# for b in band_list:
    
#     band_granules_path = glob.glob(os.path.join(b_path, 'sentinel/uncompressed/*/GRANULE/*/IMG_DATA/*_B{}.jp2'.format(b)))

#     band_granules_src = [rasterio.open(i, driver='JP2OpenJPEG') for i in band_granules_path]

#     mosaic_path = base_path / 'sentinel/merge_B{}_{}.tif'.format(b, year)

#     # # https://rasterio.readthedocs.io/en/latest/api/rasterio.merge.html
#     # band_mosaic = merge(band_granules_src, method="first", dst_path=mosaic_path, dst_kwds={"Driver": "GeoTiff"})


#     mosaic, output = merge(band_granules_src)

#     output_meta = band_granules_src[-1].meta.copy()
#     output_meta.update(
#         {"driver": "GTiff",
#             "height": mosaic.shape[1],
#             "width": mosaic.shape[2],
#             "transform": output,
#         }
#     )

#     with rasterio.open(mosaic_path, "w", **output_meta) as m:
#         m.write(mosaic)


# # convert from utm51n to wgs84?

Working on band 01
Reproject file /rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/raw_data/sentinel_shoreline/sentinel/uncompressed/S2B_MSIL1C_20170928T155019_N0205_R054_T18SVF_20170928T155933.SAFE/GRANULE/L1C_T18SVF_A002940_20170928T155933/IMG_DATA/T18SVF_20170928T155019_B01.jp2
Reproject file /rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/raw_data/sentinel_shoreline/sentinel/uncompressed/S2A_MSIL1C_20170516T154911_N0205_R054_T18SVH_20170516T160105.SAFE/GRANULE/L1C_T18SVH_A009918_20170516T160105/IMG_DATA/T18SVH_20170516T154911_B01.jp2
Reproject file /rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/raw_data/sentinel_shoreline/sentinel/uncompressed/S2A_MSIL1C_20170509T155911_N0205_R097_T18STH_20170509T160214.SAFE/GRANULE/L1C_T18STH_A009818_20170509T160214/IMG_DATA/T18STH_20170509T155911_B01.jp2
Reproject file /rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/raw_data/sentinel_shoreline/sentinel/uncompressed/S2A_MSIL1C_20170516T15